In [ ]:
! pip install git+https://github.com/m-bain/whisperx.git
!pip install light-the-torch
!ltt install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1
!pip install  git+https://github.com/hmmlearn/hmmlearn.git
!pip install  git+https://github.com/pyannote/pyannote-audio.git@develop
!pip show torch torchaudio

In [ ]:
# IF RUNNING ON GOOGLE COLAB, IF NOT COMMENT THIS CELL OUT!
# from google.colab import drive

# drive.mount('/gdrive')

# # Google Drive root directory
# root_path = "/gdrive/My Drive"

# file_path = "/gdrive/My Drive/GEHM_audio/20220916/20220916-SP05F.m4a"

Mounted at /gdrive


In [ ]:
# IF RUNNING LOCALLY, IF NOT COMMENT THIS CELL OUT

#make sure that the is in the active directory

#file_path = "patriziaTest.wav"

In [ ]:
# this took 4.19s on medium.en model, with GPU A100
# this took 4.12s on medium.en model, with GPU V100
# this took 5.03s on medium.en model, with GPU T4

# import whisperx

# device = "cuda"
# audio_file = file_path

# # transcribe with original whisper
# #model = whisperx.load_model("medium", device) # medium is twice as fast as the large one
# model = whisperx.load_model("medium", device)
# result = model.transcribe(audio_file)

# #print(result["segments"]) # before alignment

# # load alignment model and metadata
# model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)

# # align whisper output
# result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file, device)

#print(result_aligned["segments"]) # after alignment
#print(result_aligned["word_segments"]) # after alignment

In [ ]:
import os
import whisperx
import string
from google.colab import drive

drive.mount('/gdrive', force_remount=True)

data_path = '/gdrive/MyDrive/GEHM_audio/'
all_folders = os.listdir(data_path)

device = "cuda"

# transcribe with original whisper
model = whisperx.load_model("medium.en", device)

#print(all_folders)

In [ ]:
#define specific folders to process
#all_folders = ["20211007", "20220722"]
#print(all_folders)

In [ ]:
def create_single_speaker(audio_file_path, audio_file_index, audio_filename, current_global_xmin, current_global_xmax):
  new_global_xmin = current_global_xmin
  new_global_xmax = current_global_xmax

  speaker_name = audio_filename.split("-")[1].split(".")[0]

  single_speaker = '\n\titem ['+str(audio_file_index)+']:\n\t\tclass = "IntervalTier"\n\t\tname = "'+speaker_name+'"'

  result = model.transcribe(audio_file_path)
  model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
  result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file_path, device)
  local_size = len(result_aligned["word_segments"])

  output = ""
  for item_index, item in enumerate(result_aligned["word_segments"]):
    #print(item)
    word = str(item['word']).translate(str.maketrans('', '', string.punctuation))
    xmin = item['start'] if len(item)>1 else -1
    xmax = item['end'] if len(item)>1 else -1

    if xmin<current_global_xmin:
      new_global_xmin = xmin
    if xmax>current_global_xmax:
      new_global_xmax = xmax

    output += '\t\tintervals ['+str(item_index)+']:\n\t\t\txmin = '+str(xmin)+'\n\t\t\txmax = '+str(xmax)+'\n\t\t\ttext = "'+str(word)+'"\n'

  local_xmin = result_aligned["word_segments"][0]['start']
  local_xmax = result_aligned["word_segments"][-1]['end']

  single_speaker += '\n\t\txmin = '+str(local_xmin)+'\n\t\txmax = '+str(local_xmax)+'\n\t\tintervals: size = '+str(local_size)+'\n'
  single_speaker += output

  return single_speaker, new_global_xmin, new_global_xmax

In [ ]:
def generate_header(xmin, xmax, size):
  header = 'File type = "ooTextFile"\nObject class = "TextGrid"\nxmin = '+str(xmin)+'\nxmax = '+str(xmax)+'\ntier? <exists>\nsize= '+str(size)+'\nitem []:'
  return header

In [ ]:
from datetime import datetime

# this is used for naming the error list file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
failed_audios_list = []
failed_textgrid_list = []
for folder_index, folder in enumerate(all_folders):
  print("Starting with meeting "+str(folder) + ", "+str(folder_index+1)+"/"+str(len(all_folders))+".")
  audio_list = os.listdir(data_path+'/'+folder)

  global_xmin = 10000
  global_xmax = 0
  num_of_speakers = len(audio_list)

  all_speaker_results = ""
  for audio_file_index, audio_filename in enumerate(audio_list):
    print("Starting with audio file "+str(audio_filename) + ", "+str(audio_file_index+1)+"/"+str(len(audio_list))+".")
    audio_file_path = data_path+'/'+folder+'/'+audio_filename
    try:
      single_speaker_output, global_xmin, global_xmax = create_single_speaker(audio_file_path, audio_file_index, audio_filename, global_xmin, global_xmax)
      all_speaker_results += single_speaker_output
    except:
      print("Process failed for file "+str(audio_filename))
      failed_audios_list.append(audio_filename)
      continue

  final_output = generate_header(global_xmin, global_xmax, num_of_speakers)
  final_output += all_speaker_results
  try:
    f = open(data_path+'/'+folder+'/'+folder+'_Praat.TextGrid', "wb")
    f.write(final_output.encode('utf-8'))
    f.close()
  except:
    print("Failed to write TextGrid file "+folder+'_Praat.TextGrid')
    failed_textgrid_list.append(folder)
    continue

In [ ]:
f = open(data_path+'/'+'Errors list Run '+dt_string, "w")
f.write(str(failed_audios_list))
f.close()